In [2]:
import numpy as np
from sklearn.model_selection import KFold
import random    
import pandas as pd
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt

In [3]:
# UV Decomposition
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

# input the path of ratings.dat file
RT = pd.read_csv('./ml-1m/ratings.dat', engine='python', sep='::',
                 names=['user_id', 'movie_id', 'rating', 'timestamp'])

# create a kfold function to divide the data into 5 random sets for cross validation
KF = KFold(n_splits=5, shuffle=True, random_state=9)
c = 2

i = 5

# start the iteration for each of the 5 folds
for train_index, test_index in KF.split(RT):
    RT_train, RT_test = RT.loc[train_index], RT.loc[test_index]
    # create a dataframe to store all ratings as values for each movie in a coloumn with every user id as index of the rows.
    Row_df = RT_train.pivot(
        index='user_id', columns='movie_id', values='rating')
    u_mean = Row_df.mean(axis=1)
    Row_df_array = Row_df.to_numpy()
    u_mean = u_mean.to_numpy()
    # creating a normal matrix to compare to our uv matrix
    normal = Row_df_array - u_mean.reshape(-1, 1)
    N = normal
    # creating uv matrix components with u having n X d and v having d X m ( where n = number of users, m = number of movies and d = 2)
    u = np.full((normal.shape[0], 2), 1)
    v = np.full((2, normal.shape[1]), 1)
    u = u.astype(np.float32)
    v = v.astype(np.float32)
    uv = np.dot(u, v)
    print("TRAIN:", train_index, "TEST:", test_index)
    # updating u using the formula x =(Σj vsj (mrj−Σk̸=surkvkj))/Σjv^2sj
    for iterations in range(i):
        for r in range(6040):
            for s in range(c):
                sums = 0
                u_rk = u[r, :]
                v_kj = v[:, :]
                # to calculate the part of the matrices not affected by the value at index r
                u_rk_del = np.delete(u_rk, s, 0)
                v_kj_del = np.delete(v_kj, s, 0)
                v_sj = v[s, :]
                v_sj_squared = v_sj ** 2

                # create the matrix combination of u and v which would be subtracted from original matrix for error
                u_rk_v_kj = np.dot(u_rk_del, v_kj_del)
                m_rj = N[r, :]

                error = m_rj - u_rk_v_kj
                vsj_dot_er = v_sj * error
                sums = np.nansum(vsj_dot_er)
                v_sj_ssum = np.nansum((v_sj_squared) * (~np.isnan(m_rj)))
                newval_u = sums / v_sj_ssum
                u[r, s] = u[r, s] + ((newval_u - u[r, s]))
        # update v using the formula y = (Σiuir(mis−Σk̸=ruikvks))/Σiu^2ir
        for r in range(c):
            for s in range(Row_df_array.shape[1]):
                sums = 0

                u_ik = u[:, :]
                v_ks = v[:, s]
                u_ik_del = np.delete(u_ik, r, 1)

                v_ks_del = np.delete(v_ks, r, 0)
                u_ir = u[:, r]
                u_ir_squared = u_ir ** 2

                u_ik_v_ks = np.dot(u_ik_del, v_ks_del)
                m_is = N[:, s]
                error = m_is - u_ik_v_ks

                uir_dot_er = u_ir * error
                sumsv = np.nansum(uir_dot_er)
                u_ir_ssum = np.nansum(u_ir_squared * (~np.isnan(m_is)))
                newval_v = sumsv / u_ir_ssum
                v[r, s] = v[r, s] + ((newval_v - v[r, s]))

        uv = np.dot(u, v)
        # calculating MAE
        dif = uv-normal
        print("Iteration Number: ", iterations)
        dif_abs = (np.absolute(dif))
        dif_abs_0s = np.nan_to_num(dif_abs)
        dif_abs_sum = np.sum(dif_abs_0s, axis=0)
        sum_dif = dif_abs_sum.sum()
        non_0_count = np.count_nonzero(dif_abs_0s)
        MAE = sum_dif/non_0_count
        print('MAE', MAE)
        # calculating RMSE
        dif_sqr = dif ** 2
        dif_sqr_0s = np.nan_to_num(dif_sqr)
        dif_sqr_total = np.sum(dif_sqr_0s, axis=0)
        sumz = dif_sqr_total.sum()
        non_0_count_sqr = np.count_nonzero(dif_sqr_0s)
        RME = sumz / non_0_count_sqr
        rme_list = [RME]
        print('RMSE=', RME)


FileNotFoundError: [Errno 2] No such file or directory: 'ratings.dat'